### 多个输入通道
> * 每个通道都有一个卷积核，结果是所有通道卷积结果的和
![](../images/Multi-Inputs.png)
> * 输入X：$c_i\times n_h\times n_w$
> * 核W：$c_i\times k_h \times k_w$
> * 输出Y：$m_h \times m_w$
> * <font color = 'red'>$Y=\sum^{c_i}_{i=0}X_{i,:,:}\star W_{i,:,:}$</font>

### 多输出通道
> * 无论有多少输入通道，到目前为止只用到单输入通道
> * 可以有多个三维卷积核，每个核生成一个输出通道
> * 输入X：$c_i\times n_h\times n_w$
> * 核W：$c_o\times c_i\times k_h\times k_w$
> * 输出Y：$c_o\times m_h\times m_w$
> * $Y_{i,:,:}=X\star W_{i,:,:,:}$   for$i=1,...,c_o$

### 多个输入和输出通道
> * 每个输出通道可以识别特定模式
> * 输出通道核识别并组合输入中的模式

### $1\times 1$卷积层
> $k_h=k_w=1$是一个受欢迎的选择，他不识别空间模式，只是融合通道

![](../images/1x1Conv.png)
相当于输入形状为$n_h n_w\times c_i$，权重为$c_o\times c_i$的全连接层

### 二维卷积层
> * 输入X：$c_i\times n_h\times n_w$
> * 核W：$c_o\times c_i\times k_h\times k_w$
> * 偏差B：$c_o\times c_i$
> * 输出Y：$c_o\times m_h\times m_w$
> * $Y=X\star W+B$
> * 计算复杂度（浮点计算数FLOP）$O(c_i c_o k_h k_w m_h m_w)$

### 总结
> * 输出通道数时卷积层的超参数
> * 每个输入通道有独立的二维卷积核，所有通道结果相加得到一个输出通道结果
> * 每个输出通道有独立的三维卷积核

In [1]:
"""实现一下多输入通道互相关运算"""
import torch
from d2l import limutorch as d2l

def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x,k in zip(X, K))

In [11]:
X = torch.tensor([[[0.0, 1.0, 2.0],[3.0, 4.0, 5.0],[6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0],[4.0, 5.0, 6.0],[7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0],[2.0, 3.0]],[[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [12]:
"""计算多个通道的输出的互相关函数"""
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0) ##torch.stack是对内容进行拼接，与torch.cat差不多，torch.cat保持维度不变，torch.stack维度会发生变化

K = torch.stack((K, K+1, K+2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [13]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [14]:
"""1x1卷积"""
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X) ## torch.matmul是矩阵乘法，卷积进行的二维相关运算，全连接层进行的是矩阵乘法，所以这里相当于进行的是全连接层运算
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6